Desenvolva um modelo preditivo de classificação para identificar cancelamentos e utilize métricas adequadas para argumentar a efetividade do modelo;

In [ ]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
df = pd.read_csv("cancellation_prediction.csv")
df

In [ ]:
#Removendo do df as colunas que serão tratadas
df_naive = df.drop(['type', 
                    'month_arrival_date', 
                    'breakfast', 
                    'country', 
                    'reserved_room', 
                    'deposit_policy'], axis = 1)
df_naive

In [ ]:
#Transformando dtype object em int, de forma manual
type_hotel = df['type'].map({'Hotel': 0, 'Fancy Hotel' : 1})
month_arrival_date = df['month_arrival_date'].map({
                                                    'January' : 1, 
                                                    'February' : 2, 
                                                    'March' : 3, 
                                                    'April' : 4, 
                                                    'May' : 5, 
                                                    'June' : 6,
                                                    'July' : 7, 
                                                    'August' : 8, 
                                                    'September' : 9, 
                                                    'October' : 10, 
                                                    'November' : 11, 
                                                    'December' : 12,})
breakfast = df['breakfast'].map({True : 0, False : 1})
deposit_policy = df['deposit_policy'].map({'No Deposit': 0, 
                                                 'Refundable' : 1, 
                                                 'Non Refund' : 2})
transform_map = pd.concat([type_hotel, month_arrival_date, breakfast, deposit_policy], axis = 1)
transform_map

In [ ]:
#transformando através do get_dummies
Transform_dummies = pd.get_dummies(df.drop(['type', 
                                      'cancellation', 
                                      'days_between_booking_arrival',
                                     'year_arrival_date',
                                     'month_arrival_date',
                                     'week_number_arrival_date',
                                     'day_of_month_arrival_date',
                                     'num_weekend_nights',
                                     'num_workweek_nights',
                                     'num_adults',
                                     'num_children',
                                     'num_babies',
                                     'breakfast',
                                     'market_segment',
                                     'distribution_channel',
                                     'repeated_guest',
                                     'num_previous_cancellations',
                                     'num_previous_stays',
                                     'changes_between_booking_arrival',
                                     'deposit_policy',
                                     'id_travel_agency_booking',
                                     'id_person_booking',
                                     'customer_type',
                                     'avg_price',
                                     'required_car_parking_spaces',
                                     'total_of_special_requests'], axis = 1))
Transform_dummies

In [ ]:
#juntando os dataframes tratados
df_final = pd.concat([df_naive, transform_map, Transform_dummies], axis = 1)
#removendo colunas 'id_travel_agency_booking' e 'id_person_booking', removendo nan.
df_final = df_final.drop(['id_travel_agency_booking', 'id_person_booking'], axis = 1).dropna()
df_final

#modelo árvore de decisão 

In [ ]:
#Divisão em inputs e outputs
X = df_final.drop('cancellation', axis = 1)
y = df_final['cancellation']

In [ ]:
#normalizando imouts 
norm = StandardScaler()
x_norm = norm.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state = 123)

treino teste

In [ ]:
#decision tree
dtc= DecisionTreeClassifier(criterion='entropy', random_state=42)

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
dtc.feature_importances_

In [ ]:
predict_dtc = dtc.predict(X_test)
predict_dtc

Validação

In [ ]:
#mariz de confusão
r_confusion_matrix = confusion_matrix(y_test, predict_dtc)

In [ ]:
#acurácia
r_accuracy = accuracy_score(y_test, predict_dtc)

In [ ]:
#precisão
r_precision = precision_score(y_test, predict_dtc)

In [ ]:
#Recall
r_recall = recall_score(y_test, predict_dtc)

Ao realizar a validação cruzada do modelo de classificação, discuta sobre as diferenças entre utilizar uma separação entre treino e teste aleatória e uma separação temporal (Ex: treino em 2015 e 2016 e validação em 2017). Os resultados são diferentes? Qual o mais indicado?

In [ ]:
#Montando treino-texte separado por ano
df_2015 = df_final.loc[df_final['year_arrival_date'] == 2015]
df_2016 = df_final.loc[df_final['year_arrival_date'] == 2016]
df_2017 = df_final.loc[df_final['year_arrival_date'] == 2017]
#juntando dfs 2015 e 2016
df_15_16 = pd.concat([df_2015, df_2016], axis = 0)
#Divisão em inputs e outputs
X_15_16 = df_15_16.drop('cancellation', axis = 1)
y_15_16 = df_15_16['cancellation']

X_17 = df_2017.drop('cancellation', axis = 1)
y_17 = df_2017['cancellation']

In [ ]:
norm = StandardScaler()
x_norm_15_16 = norm.fit_transform(X_15_16)

In [ ]:
#decision tree
dtc= DecisionTreeClassifier(criterion='entropy', random_state=42)

In [ ]:
dtc.fit(X_15_16, y_15_16)

In [ ]:
predict_dtc = dtc.predict(X_17)
predict_dtc

In [ ]:
#mariz de confusão
t_confusion_matrix = confusion_matrix(y_17, predict_dtc)

In [ ]:
#acurácia
t_accuracy = accuracy_score(y_17, predict_dtc)

In [ ]:
#precisão
t_precision = precision_score(y_17, predict_dtc)

In [ ]:
#Recall
t_recall = recall_score(y_17, predict_dtc)

In [ ]:
#comparando os métodos de separação
print('Tratamento temporal - Matriz de confusão', t_confusion_matrix)
print('Separação randômica - Matriz de confusão', r_confusion_matrix) 
print('Tratamento temporal - Acurácia', t_accuracy) 
print('Separação randômica - Acurácia', r_accuracy)
print('Tratamento temporal - Precisão', t_precision)
print('Separação randômica - Precisão', r_precision)
print('Tratamento temporal - recall', t_recall)
print('Separação randômica - recall', r_recall)

O método de separação da base de forma aleatória utilizando o train_test_split apresentou melhor resultado.
Na matriz de confusão, apesar do valor de verdadeiro positivo ser maior no método utilizando a separação temporal, entretanto, avaliando os valores de verdadeiro negativo, falso negativo e falso positivo, o método de separação da base de dados de forma aleatória mostra-se mais eficiente.
Tanto na acurácia, quanto na precisão e recall, o método de separação da base de forma aleatória também obteve melhor resultado, com 84 %, 79 % e 79 % de acerto respectivamente.
O resultado pode estar enviesado, já que o modelo elegido foi a arvore de decisão, e, o ponto fraco desse modelo é exatamente apresentar overfitting. A separação da base de dados por ano pode apresentar diferenças no comportamento dos clientes, como sazonalidade e clima, que não são contemplados nas características da base. O qual pode explicar o pior desempenho.